# Install and Import packages

In [1]:
#!install_package_python37.sh add xlrd
#!install_package_python37.sh add nltk==3.7
#!install_package_python37.sh add python-dotenv
#!install_package_python37.sh add scikit-learn==0.21.1
#!install_package_python37.sh add pyjwt
#!install_package_python37.sh add boxsdk
#!install_package_python37.sh add openpyxl
#!install_package_python37.sh add sendgrid

In [2]:
# import necessary packages
import joblib
import os
import re
import string
from datetime import datetime, timezone,timedelta
import logging
from logging.handlers import TimedRotatingFileHandler
from boxsdk import Client, JWTAuth
import chardet
from dotenv import load_dotenv
import io
import numpy as np
import pandas as pd
import json
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')
import nltk
#nltk.download('popular')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
import sklearn.metrics as metrics
from nltk.tag import pos_tag
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, cross_val_predict
import sendgrid
from sendgrid import SendGridAPIClient
load_dotenv(".env")
sg = sendgrid.SendGridAPIClient(os.environ.get('api_key'))
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/irpa3/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/irpa3/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/irpa3/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/irpa3/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/irpa3/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/irpa3/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!

True

In [3]:
# Set up logger

LOGFILE = "logs/job1.log"

logger = logging.getLogger("dailylogs")

# set logging level : INFO, DEBUG, WARNING or ERROR
logger.setLevel(logging.DEBUG)


# Create TimedRotatingFileHandler with log file name
# It will create a new log file each day at midnight
handler = TimedRotatingFileHandler(LOGFILE, when="midnight", interval=1)

# This is the format in which logs will be displayed in log file
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# assign the formatter and suffix to file_handler object
# suffix will be added to each file
handler.setFormatter(formatter)
handler.suffix = "%Y%m%d"


# add the handler to logger
logger.addHandler(handler)

In [4]:
start_time = time.time()
logger.debug('New Job has started ....')

In [5]:
def preprocess_text(message):

    #stopwords
    new_stopwords=['hi','hello','team','thanks','hey','regards','please','jira','com','ext','image','capture']
    stpwrd = nltk.corpus.stopwords.words('english')
    stpwrd.extend(new_stopwords)
    # 1. Init Lemmatizer
    lemmatizer = WordNetLemmatizer()
    message=message.lower()
    #removing the numerical values and working only with text values
    message = re.sub('[^a-zA-Z]', " ", message)
    #lowering and removing punctuation
    message = re.sub(r'[^\w\s]','',message)
    #removing the stopwords
    message = ' '.join([word for word in message.split() if word not in stpwrd and len(word)>1])
    #lemmatizing the text
    message =  " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(message) if w not in string.punctuation])
    return message

# Automated email notification

In [6]:
message_exception_1 = {
    'personalizations': [
        {
    "to": [{
            "email": "amoham117@ext.uber.com"
        }, {
            "email": "nbandi@ext.uber.com"
        },
        {
            "email": "ssethu@ext.uber.com"
        },
         {
            "email": "rkunta@ext.uber.com"
        },
        {
            "email": "vbanap@ext.uber.com"
        },
        {
            "email": "pyagat@ext.uber.com"
        },
        {
            "email": "rsujay@ext.uber.com"
        }],
            'subject': 'L1Triaging_MLModelAutoLearning Failure Notification'
        }
    ],
    'from': {
        'email': 'ia-coe-support-group@uber.com'
    },
    'content': [
        {
            'type': 'text/html',
            "value": "<html><head></head><body>Hello!<br><br>L1Triaging_MLModelAutoLearning automation has encountered an Error And the team Is currently analyzing the issue. We will follow up shortly With our Next steps For resolution And Let you know When the issue Is triaged.Please reach Out To ia-coe-support-group@uber.com For any questions Or concerns.<br><br>Error Message: Please provide corresponding Report3.xlsx with proper columns <br><br>Kindly check And rerun the process.<br><br><br>Thanks,<br><b>Intelligent Automation COE Support Group<b></body></html>"
  
        }
    ]
}
message_exception_2 = {
    'personalizations': [
        {
    "to": [{
            "email": "amoham117@ext.uber.com"
        }, {
            "email": "nbandi@ext.uber.com"
        },
        {
            "email": "ssethu@ext.uber.com"
        },
         {
            "email": "rkunta@ext.uber.com"
        },
        {
            "email": "vbanap@ext.uber.com"
        },
        {
            "email": "pyagat@ext.uber.com"
        },
        {
            "email": "rsujay@ext.uber.com"
        }],
            'subject': 'L1Triaging_MLModelAutoLearning Failure Notification'
        }
    ],
    'from': {
        'email': 'ia-coe-support-group@uber.com'
    },
    'content': [
        {
            'type': 'text/html',
            "value": "<html><head></head><body>Hello!<br><br>L1Triaging_MLModelAutoLearning automation has encountered an Error And the team Is currently analyzing the issue. We will follow up shortly With our Next steps For resolution And Let you know When the issue Is triaged.Please reach Out To ia-coe-support-group@uber.com For any questions Or concerns.<br><br>Error Message: No file present in input location <br><br>Kindly check And rerun the process.<br><br><br>Thanks,<br><b>Intelligent Automation COE Support Group<b></body></html>"

                
        }
    ]
}

# cicd data import from box

In [7]:
config = JWTAuth.from_settings_file('Tax_Auto_Config.json')
client = Client(config)
INPUT_FOLDER_ID = '170368566025'
ARCHIVE_FOLDER_ID = '170364762149'

In [8]:
logger.debug('Read the file from BOX location... ')
folder = client.folder(folder_id=INPUT_FOLDER_ID)
items = folder.get_items()
# make a list of iten name and item id of all items
files = [(item.name, item.id) for item in items]
datetime_str = datetime.now().strftime("%d_%m_%Y")
#print(datetime_str)
FILENAME = 'Report3.xlsx'
FILENAME = datetime_str+"_"+FILENAME
print(files)
# sort it by filename
files.sort(reverse=True, key=lambda x: x[0])

try:
    if len(files)>=1:
        for i, f in enumerate(files):
            # get id of the latest file (based on timestamp)
            file_id = f[1]   
           # print(f[0])
            if f[0]==FILENAME:
                # read the content and convert it into Pandas DataFrame
                with open(FILENAME, 'wb') as open_file:
                    client.file(file_id).download_to(open_file)
                    open_file.close()
                #data_cicd = pd.read_excel(io.StringIO(file_info.decode('utf-16')),usecols=['TicketSummary','TicketDescription','Corrected Component','Corrected Tier Type 1','Corrected Tier Type 2'])
                data_cicd_latest = pd.read_excel(open(FILENAME, 'rb'),engine='openpyxl',usecols=['TicketSummary','TicketDescription','Corrected Component','Corrected Tier Type 1','Corrected Tier Type 2'])
                logger.debug('CICD File {} has been downloaded from BOX with shape {}...'.format( f[0],data_cicd_latest.shape))
            else:
                logger.debug('Please provide {} with columns: {}'.format(FILENAME,data_cicd.columns))
                response=sg.client.mail.send.post(request_body=message_exception_1)
                
    else:
        logger.error('No file present in input location...')
        response=sg.client.mail.send.post(request_body=message_exception_2)
        
except Exception as e:
    print('kk')
    logger.error('Exception details::{}'.format(e), exc_info=True)
   ## response=sg.client.mail.send.post(request_body=message_exception_1)

[('12_12_2022_Report3.xlsx', '1088505411021')]


In [9]:
data_cicd=pd.read_csv('./data/data_cicd_latest.csv',index_col=0)
data_cicd=data_cicd.append(data_cicd_latest)
logger.debug('Incremental CICD data has been added with shape {}'.format(data_cicd.shape))
data_cicd.to_csv('./data/data_cicd_latest.csv')

In [10]:
data_cicd_component=data_cicd.dropna(subset=['Corrected Component'])
data_cicd_component['combined_text'] = data_cicd_component[['TicketSummary','TicketDescription']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
# apply data preprocessing steps on the prepared column
data_cicd_component['processed_text'] = data_cicd_component['combined_text'].map(lambda s:preprocess_text(s)) 
data_cicd_component=data_cicd_component.drop_duplicates(subset=['processed_text','Corrected Component'],keep='first')
data_cicd_component = data_cicd_component.reset_index(drop=True)
X_cicd_component= data_cicd_component[['TicketSummary','TicketDescription','processed_text']]
y_cicd_component= data_cicd_component['Corrected Component']
logger.debug('Input data size of component: {}'.format(len(X_cicd_component)))
data_cicd_tier_type_1=data_cicd.dropna(subset=['Corrected Tier Type 1'])
data_cicd_tier_type_1['combined_text'] = data_cicd_tier_type_1[['TicketSummary','TicketDescription']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
# apply data preprocessing steps on the prepared column
data_cicd_tier_type_1['processed_text'] = data_cicd_tier_type_1['combined_text'].map(lambda s:preprocess_text(s)) 
data_cicd_tier_type_1=data_cicd_tier_type_1.drop_duplicates(subset=['processed_text','Corrected Tier Type 1'],keep='first')
data_cicd_tier_type_1 = data_cicd_tier_type_1.reset_index(drop=True)
X_cicd_tier_type_1= data_cicd_tier_type_1[['TicketSummary','TicketDescription','processed_text']]
y_cicd_tier_type_1= data_cicd_tier_type_1['Corrected Tier Type 1']
logger.debug('Input data size of tier_type_1: {}'.format(len(X_cicd_tier_type_1)))
data_cicd_tier_type_2=data_cicd.dropna(subset=['Corrected Tier Type 2'])
data_cicd_tier_type_2['combined_text'] = data_cicd_tier_type_2[['TicketSummary','TicketDescription']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
# apply data preprocessing steps on the prepared column
data_cicd_tier_type_2['processed_text'] = data_cicd_tier_type_2['combined_text'].map(lambda s:preprocess_text(s)) 
data_cicd_tier_type_2=data_cicd_tier_type_2.drop_duplicates(subset=['processed_text','Corrected Tier Type 2'],keep='first')
data_cicd_tier_type_2 = data_cicd_tier_type_2.reset_index(drop=True)
X_cicd_tier_type_2= data_cicd_tier_type_2[['TicketSummary','TicketDescription','processed_text']]
y_cicd_tier_type_2= data_cicd_tier_type_2['Corrected Tier Type 2']
logger.debug('Input data size of tier_type_2: {}'.format(len(X_cicd_tier_type_2)))

In [11]:
#read input from historical data into dataframe
df_1 = pd.read_csv('./data/historical_data.csv')
df_1['TicketSummary']=df_1['TicketSummary'].astype(str)
df_1['TicketDescription']=df_1['TicketDescription'].astype(str)
#df_1.drop(['Unnamed: 0','Unnamed: 5','combined_text','processed_text'],axis=1)
#choose sample data from entire data
data_df = df_1.sample(frac=1, random_state=42)
data_df['combined_text'] = data_df[['TicketSummary','TicketDescription']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
# apply data preprocessing steps on the prepared column
data_df['processed_text'] = data_df['combined_text'].map(lambda s:preprocess_text(s)) 
data_df=data_df.drop_duplicates(subset=['processed_text','Corrected Component','Corrected Tier Type 1','Corrected Tier Type 2'],keep='first')
#data_df.rename(columns={'Summary': 'TicketSummary', 'Description': 'TicketDescription','Updated Component': 'Corrected Component', 'Tier Type 1': 'Corrected Tier Type 1', 'Tier Type 2': 'Corrected Tier Type 2'}, inplace=True)
#data_df.to_csv('./data/historical_data.csv')
logger.debug('Input data size of historical data: {}'.format(data_df.shape))

In [12]:
data_df.head()

,Unnamed: 0,TicketSummary,TicketDescription,Corrected Component,Corrected Tier Type 1,Corrected Tier Type 2,combined_text,processed_text
216,13148,Difference in Historical Balance,"Hi Team,\n\n\n\n\nWe have observed that there ...",Oracle Payables,Oracle_ap,BLANK,"Difference in Historical Balance Hi Team,\n\n\...",difference historical balance observed change ...
13066,12042,Coupa Access,"Hi Finance Support Team,\n\n\nI work on the ch...",Access Requests,Coupa_access,Coupa_access_request,"Coupa Access Hi Finance Support Team,\n\n\nI w...",coupa access finance support work channel mark...
12354,15917,[OTM Oracle] Please add Uber Finance role to b...,"Dears,\n\n\nPlease add the UBER_FINANCE role t...",Oracle Transport Management,OTM,OTM,[OTM Oracle] Please add Uber Finance role to b...,otm oracle add uber finance role user dear add...
37411,27649,Access for HSBC & ICICI Bank statement,"Hi Team,\n\n\n\nPlease could you provide acces...",Kyriba,Kyriba,Kyriba_report_access,Access for HSBC & ICICI Bank statement Hi Team...,access hsbc icici bank statement could provide...
24280,25266,Unable to find the approver name in Oralce.,"Hi Team,\n\n\nHope you're doing well!\n\n\n\nW...",Access Requests,Oracle_sysadmin,BLANK,Unable to find the approver name in Oralce. Hi...,unable find approver name oralce hope well abl...


In [13]:
X = data_df[['TicketSummary','TicketDescription','processed_text']]
y_component= data_df['Corrected Component']
y_tier_type_1= data_df['Corrected Tier Type 1']
y_tier_type_2= data_df['Corrected Tier Type 2']
# split the cicd data into train and test 
X_train_component, X_test_component, y_train_component, y_test_component = train_test_split(X, y_component, test_size = .20, random_state = 42)
X_train_tier_type_1, X_test_tier_type_1, y_train_tier_type_1, y_test_tier_type_1 = train_test_split(X, y_tier_type_1, test_size = .20, random_state = 42)
X_train_tier_type_2, X_test_tier_type_2, y_train_tier_type_2, y_test_tier_type_2 = train_test_split(X, y_tier_type_2, test_size = .20, random_state = 42)

# component training part

In [14]:
X_train_final_component = X_train_component.append(X_cicd_component)
X_test_final_component = X_test_component
y_train_final_component = y_train_component.append(y_cicd_component)
y_test_final_component = y_test_component
logger.debug('Training data size of component: {} {}'.format(len(X_train_final_component),len(y_train_final_component)))
logger.debug('Test data size of component: {} {}'.format(len(X_test_final_component),len(y_test_final_component)))
model_component= Pipeline([('vect', CountVectorizer(max_df=0.80)),
       ('tfidf', TfidfTransformer()),
       #('clf', RandomForestClassifier(class_weight='balanced'))])
       ('mnb', MultinomialNB(fit_prior=False,alpha=0.05))])
# perform model training
model_component.fit(X_train_final_component['processed_text'].values, y_train_final_component.values)
logger.debug('Model fit successfully completed for component model...')

In [15]:
# model prediction
result1=X_test_final_component
y_pred = model_component.predict(X_test_final_component['processed_text'].values)

result1['original_cat_component']= y_test_final_component
result1['predicted_cat_component'] = y_pred

result1['prediction_cat_confscore_component'] = np.round_(np.max(model_component.predict_proba(X_test_final_component['processed_text'].values), axis=1), decimals=2)

#
output = {'accuracy': accuracy_score(y_pred,y_test_final_component),'precision_score':precision_score(y_pred,y_test_final_component,average='weighted'),'recall_score':recall_score(y_pred,y_test_final_component,average='weighted')
,'f1_score':f1_score(y_pred,y_test_final_component,average='weighted')}

result1['confusion_matrix'] = str(output)
print(result1['confusion_matrix'] [5:6].values)

["{'accuracy': 0.7516113630938172, 'precision_score': 0.7774055232286262, 'recall_score': 0.7516113630938172, 'f1_score': 0.7605012122257254}"]


In [16]:
X_train_final_component.shape,X_test_final_component.shape

((33801, 3), (8378, 7))

In [17]:
logger.info('confusion_matrix for component:'+result1['confusion_matrix'] [5:6].values)

# tier type 1 training

In [18]:
X_train_final_tier_type_1 = X_train_tier_type_1.append(X_cicd_tier_type_1)
X_test_final_tier_type_1 = X_test_tier_type_1
y_train_final_tier_type_1 = y_train_tier_type_1.append(y_cicd_tier_type_1)
y_test_final_tier_type_1 = y_test_tier_type_1
logger.debug('Training data size of tier type 1: {} {}'.format(len(X_train_final_tier_type_1),len(y_train_final_tier_type_1)))
logger.debug('Test data size of tier type 1: {} {}'.format(len(X_test_final_tier_type_1),len(y_test_final_tier_type_1)))
model_tier_type_1= Pipeline([('vect', CountVectorizer(max_df=0.80)),
       ('tfidf', TfidfTransformer()),
       #('clf', RandomForestClassifier(class_weight='balanced'))])
       ('mnb', MultinomialNB(fit_prior=False,alpha=0.05))])
# perform model training
model_tier_type_1.fit(X_train_final_tier_type_1['processed_text'].values, y_train_final_tier_type_1.values)
logger.debug('Model fit successfully completed for tier type 1 model...')

In [19]:
# model prediction
result2=X_test_final_tier_type_1
y_pred = model_tier_type_1.predict(X_test_final_tier_type_1['processed_text'].values)

result2['original_cat_tier_type_1']= y_test_final_tier_type_1
result2['predicted_cat_tier_type_1'] = y_pred

result2['prediction_cat_confscore_tier_type_1'] = np.round_(np.max(model_tier_type_1.predict_proba(X_test_final_tier_type_1['processed_text'].values), axis=1), decimals=2)

#
output = {'accuracy': accuracy_score(y_pred,y_test_final_tier_type_1),'precision_score':precision_score(y_pred,y_test_final_tier_type_1,average='weighted'),'recall_score':recall_score(y_pred,y_test_final_tier_type_1,average='weighted')
,'f1_score':f1_score(y_pred,y_test_final_tier_type_1,average='weighted')}

result2['confusion_matrix'] = str(output)
print(result2['confusion_matrix'] [5:6].values)

["{'accuracy': 0.7346622105514442, 'precision_score': 0.7752703923594014, 'recall_score': 0.7346622105514442, 'f1_score': 0.7493648837351858}"]


In [20]:
logger.info('confusion_matrix for tier type 1:'+result2['confusion_matrix'] [5:6].values)

# tier type 2 training

In [21]:
X_train_final_tier_type_2 = X_train_tier_type_2.append(X_cicd_tier_type_2)
X_test_final_tier_type_2 = X_test_tier_type_2
y_train_final_tier_type_2 = y_train_tier_type_2.append(y_cicd_tier_type_2)
y_test_final_tier_type_2 = y_test_tier_type_2
logger.debug('Training data size of tier type 2: {} {}'.format(len(X_train_final_tier_type_2),len(y_train_final_tier_type_2)))
logger.debug('Test data size of tier type 2: {} {}'.format(len(X_test_final_tier_type_2),len(y_test_final_tier_type_2)))
model_tier_type_2= Pipeline([('vect', CountVectorizer(max_df=0.80)),
       ('tfidf', TfidfTransformer()),
       #('clf', RandomForestClassifier(class_weight='balanced'))])
       ('mnb', MultinomialNB(fit_prior=False,alpha=0.05))])
# perform model training
model_tier_type_2.fit(X_train_final_tier_type_2['processed_text'].values, y_train_final_tier_type_2.values)
logger.debug('Model fit successfully completed for tier type 2 model...')

In [22]:
# model prediction
result3=X_test_final_tier_type_2
y_pred = model_tier_type_2.predict(X_test_final_tier_type_2['processed_text'].values)

result3['original_cat_tier_type_2']= y_test_final_tier_type_2
result3['predicted_cat_tier_type_2'] = y_pred

result3['prediction_cat_confscore_tier_type_2'] = np.round_(np.max(model_tier_type_2.predict_proba(X_test_final_tier_type_2['processed_text'].values), axis=1), decimals=2)

#
output = {'accuracy': accuracy_score(y_pred,y_test_final_tier_type_2),'precision_score':precision_score(y_pred,y_test_final_tier_type_2,average='weighted'),'recall_score':recall_score(y_pred,y_test_final_tier_type_2,average='weighted')
,'f2_score':f1_score(y_pred,y_test_final_tier_type_2,average='weighted')}

result3['confusion_matrix'] = str(output)
print(result3['confusion_matrix'] [5:6].values)

["{'accuracy': 0.6008593936500358, 'precision_score': 0.6955837668608098, 'recall_score': 0.6008593936500358, 'f2_score': 0.6336917346533826}"]


In [23]:
logger.info('confusion_matrix for tier type 2:'+result3['confusion_matrix'] [5:6].values)

# model saving 

In [24]:
import pickle
import joblib
import datetime
model_path_component='./model/component'
model_path_tier_type_1='./model/tier_type_1'
model_path_tier_type_2='./model/tier_type_2'
# save the model to disk
filename_primary= 'finalized_model.sav'
component_path = os.path.join(model_path_component, filename_primary) 
tier_type_1_path = os.path.join(model_path_tier_type_1, filename_primary) 
tier_type_2_path = os.path.join(model_path_tier_type_2, filename_primary) 
pickle.dump(model_component, open(component_path, 'wb'))
pickle.dump(model_tier_type_1, open(tier_type_1_path, 'wb'))
pickle.dump(model_tier_type_2, open(tier_type_2_path, 'wb'))

# Data saving

In [25]:
X_train_component_data=X_train_final_component.drop(['processed_text'],axis=1)
X_test_component_data=X_test_final_component.drop(['processed_text','original_cat_component','predicted_cat_component','prediction_cat_confscore_component','confusion_matrix'],axis=1)
print(X_train_final_component.shape,X_test_final_component.shape)
X_train_component_data['component']=y_train_final_component
X_train_component_data['label']='train'
X_test_component_data['component']=y_test_final_component
X_test_component_data['label']='test'
component_train_test=X_train_component_data.append(X_test_component_data)

X_train_tier_type_1_data=X_train_final_tier_type_1.drop(['processed_text'],axis=1)
X_test_tier_type_1_data=X_test_final_tier_type_1.drop(['processed_text','original_cat_tier_type_1','predicted_cat_tier_type_1','prediction_cat_confscore_tier_type_1','confusion_matrix'],axis=1)
print(X_train_tier_type_1_data.shape,X_test_tier_type_1_data.shape)
X_train_tier_type_1_data['component']=y_train_final_tier_type_1
X_train_tier_type_1_data['label']='train'
X_test_tier_type_1_data['component']=y_test_final_tier_type_1
X_test_tier_type_1_data['label']='test'
tier_type_1_train_test=X_train_tier_type_1_data.append(X_test_tier_type_1_data)


X_train_tier_type_2_data=X_train_final_tier_type_2.drop(['processed_text'],axis=1)
X_test_tier_type_2_data=X_test_final_tier_type_2.drop(['processed_text','original_cat_tier_type_2','predicted_cat_tier_type_2','prediction_cat_confscore_tier_type_2','confusion_matrix'],axis=1)
print(X_train_tier_type_2_data.shape,X_test_tier_type_2_data.shape)
X_train_tier_type_2_data['component']=y_train_final_tier_type_2
X_train_tier_type_2_data['label']='train'
X_test_tier_type_2_data['component']=y_test_final_tier_type_2
X_test_tier_type_2_data['label']='test'
tier_type_2_train_test=X_train_tier_type_2_data.append(X_test_tier_type_2_data)

data_path_component='./data/component'
data_path_tier_type_1='./data/tier_type_1'
data_path_tier_type_2='./data/tier_type_2'
filename_primary_data= 'dftraintest.csv'

component_path_data = os.path.join(data_path_component, filename_primary_data) 
tier_type_1_path_data = os.path.join(data_path_tier_type_1, filename_primary_data) 
tier_type_2_path_data = os.path.join(data_path_tier_type_2, filename_primary_data) 

component_train_test.to_csv(component_path_data)
tier_type_1_train_test.to_csv(tier_type_1_path_data)
tier_type_2_train_test.to_csv(tier_type_2_path_data)

(33801, 3) (8378, 7)
(33848, 2) (8378, 2)
(33930, 2) (8378, 2)


# component model upload

In [26]:
# Folder ID to upload the data
#FOLDER_ID = '164616436488'
OUTPUT_FOLDER_ID = '170542616761'

# Create Filename based on timestamp
FILENAME = 'finalized_model.sav'
#datetime_str = datetime.now(timezone.utc).strftime("%Y%m%d%H%M%S")
# print(datetime_str)

#FILENAME = FILENAME + "_" + datetime_str+".csv"

#print(FILENAME)


# establish a connection for BOX
config = JWTAuth.from_settings_file('Tax_Auto_Config.json')
client = Client(config)
# fetch all the items from the folder
folder = client.folder(folder_id=OUTPUT_FOLDER_ID)
items = folder.get_items()
flag = False

logger.debug('Uploading component model in BOX....')

for item in items:
    # if file name exists, update the content
    if item.name == FILENAME:
        chunked_uploader = client.file(item.id).get_chunked_uploader(component_path)
        updated_file = chunked_uploader.start()
        print(f'File "{updated_file.name}" updated to Box with file ID {updated_file.id}')
        logger.debug(f'File "{updated_file.name}" updated to Box with file ID {updated_file.id}')
        flag = True
        break

# if file doesn not exists upload the file
if not flag:
    uploaded_file = folder.upload(component_path)
    print('File "{0}" has been uploaded'.format(uploaded_file.name))
    logger.debug(f'File "{uploaded_file.name}" uploaded to Box with file ID {uploaded_file.id}')

File "finalized_model.sav" has been uploaded


# component data upload

In [27]:
# Folder ID to upload the data
OUTPUT_FOLDER_ID = '170542616761'
# Create Filename based on timestamp
FILENAME ='dftraintest.csv'

# establish a connection for BOX
config = JWTAuth.from_settings_file('Tax_Auto_Config.json')
client = Client(config)
# fetch all the items from the folder
folder = client.folder(folder_id=OUTPUT_FOLDER_ID)
items = folder.get_items()
flag = False

logger.debug('Uploading component data in BOX....')

for item in items:
    # if file name exists, update the content
    if item.name == FILENAME:
        chunked_uploader = client.file(item.id).get_chunked_uploader(component_path_data)
        updated_file = chunked_uploader.start()
        print(f'File "{updated_file.name}" updated to Box with file ID {updated_file.id}')
        logger.debug(f'File "{updated_file.name}" updated to Box with file ID {updated_file.id}')
        flag = True
        break

# if file doesn not exists upload the file
if not flag:
    uploaded_file = folder.upload(component_path_data)
    print('File "{0}" has been uploaded'.format(uploaded_file.name))
    logger.debug(f'File "{uploaded_file.name}" uploaded to Box with file ID {uploaded_file.id}')

File "dftraintest.csv" has been uploaded


# tier type 1 model upload

In [28]:
# Folder ID to upload the data
#FOLDER_ID = '164616436488'
OUTPUT_FOLDER_ID = '172902610098'
# Create Filename based on timestamp
FILENAME = 'finalized_model.sav'
#datetime_str = datetime.now(timezone.utc).strftime("%Y%m%d%H%M%S")
# print(datetime_str)
#FILENAME = FILENAME + "_" + datetime_str+".csv"
#print(FILENAME)
# establish a connection for BOX
config = JWTAuth.from_settings_file('Tax_Auto_Config.json')
client = Client(config)
# fetch all the items from the folder
folder = client.folder(folder_id=OUTPUT_FOLDER_ID)
items = folder.get_items()
flag = False

logger.debug('Uploading tier type 1 model in BOX....')

for item in items:
    # if file name exists, update the content
    if item.name == FILENAME:
        chunked_uploader = client.file(item.id).get_chunked_uploader(tier_type_1_path)
        updated_file = chunked_uploader.start()
        print(f'File "{updated_file.name}" updated to Box with file ID {updated_file.id}')
        logger.debug(f'File "{updated_file.name}" updated to Box with file ID {updated_file.id}')
        flag = True
        break

# if file doesn not exists upload the file
if not flag:
    uploaded_file = folder.upload(tier_type_1_path)
    print('File "{0}" has been uploaded'.format(uploaded_file.name))
    logger.debug(f'File "{uploaded_file.name}" uploaded to Box with file ID {uploaded_file.id}')

File "finalized_model.sav" has been uploaded


# tier type 1 data upload

In [29]:
# Folder ID to upload the data
OUTPUT_FOLDER_ID = '172902610098'
# Create Filename based on timestamp
FILENAME ='dftraintest.csv'

# establish a connection for BOX
config = JWTAuth.from_settings_file('Tax_Auto_Config.json')
client = Client(config)
# fetch all the items from the folder
folder = client.folder(folder_id=OUTPUT_FOLDER_ID)
items = folder.get_items()
flag = False

logger.debug('Uploading tier type 1 data in BOX....')

for item in items:
    # if file name exists, update the content
    if item.name == FILENAME:
        chunked_uploader = client.file(item.id).get_chunked_uploader(tier_type_1_path_data)
        updated_file = chunked_uploader.start()
        logger.debug(f'File "{updated_file.name}" updated to Box with file ID {updated_file.id}')
        flag = True
        break

# if file doesn not exists upload the file
if not flag:
    uploaded_file = folder.upload(tier_type_1_path_data)
    print('File "{0}" has been uploaded'.format(uploaded_file.name))
    logger.debug(f'File "{uploaded_file.name}" uploaded to Box with file ID {uploaded_file.id}')

File "dftraintest.csv" has been uploaded


# tier type 2 model upload

In [30]:
# Folder ID to upload the data
#FOLDER_ID = '164616436488'
OUTPUT_FOLDER_ID = '172905132682'
# Create Filename based on timestamp
FILENAME = 'finalized_model.sav'
#datetime_str = datetime.now(timezone.utc).strftime("%Y%m%d%H%M%S")
# print(datetime_str)
#FILENAME = FILENAME + "_" + datetime_str+".csv"
#print(FILENAME)
# establish a connection for BOX
config = JWTAuth.from_settings_file('Tax_Auto_Config.json')
client = Client(config)
# fetch all the items from the folder
folder = client.folder(folder_id=OUTPUT_FOLDER_ID)
items = folder.get_items()
flag = False

logger.debug('Uploading tier type 2 model in BOX....')

for item in items:
    # if file name exists, update the content
    if item.name == FILENAME:
        chunked_uploader = client.file(item.id).get_chunked_uploader(tier_type_2_path)
        updated_file = chunked_uploader.start()
        print(f'File "{updated_file.name}" updated to Box with file ID {updated_file.id}')
        logger.debug(f'File "{updated_file.name}" updated to Box with file ID {updated_file.id}')
        flag = True
        break

# if file doesn not exists upload the file
if not flag:
    uploaded_file = folder.upload(tier_type_2_path)
    print('File "{0}" has been uploaded'.format(uploaded_file.name))
    logger.debug(f'File "{uploaded_file.name}" uploaded to Box with file ID {uploaded_file.id}')

File "finalized_model.sav" has been uploaded


# tier type 2 data upload

In [31]:
# Folder ID to upload the data
OUTPUT_FOLDER_ID = '172905132682'
# Create Filename based on timestamp
FILENAME ='dftraintest.csv'

# establish a connection for BOX
config = JWTAuth.from_settings_file('Tax_Auto_Config.json')
client = Client(config)
# fetch all the items from the folder
folder = client.folder(folder_id=OUTPUT_FOLDER_ID)
items = folder.get_items()
flag = False

logger.debug('Uploading tier type 2 data in BOX....')

for item in items:
    # if file name exists, update the content
    if item.name == FILENAME:
        chunked_uploader = client.file(item.id).get_chunked_uploader(tier_type_2_path_data)
        updated_file = chunked_uploader.start()
        print(f'File "{updated_file.name}" updated to Box with file ID {updated_file.id}')
        logger.debug(f'File "{updated_file.name}" updated to Box with file ID {updated_file.id}')
        flag = True
        break

# if file doesn not exists upload the file
if not flag:
    uploaded_file = folder.upload(tier_type_2_path_data)
    print('File "{0}" has been uploaded'.format(uploaded_file.name))
    logger.debug(f'File "{uploaded_file.name}" uploaded to Box with file ID {uploaded_file.id}')

File "dftraintest.csv" has been uploaded


# hyperparameter optimization

import numpy as np
from sklearn.model_selection import GridSearchCV
grid_params = {
  'mnb__alpha': [0.01,0.05,0.1,0.2,0.3,0.4,0.5],
}
clf = GridSearchCV(model_component, grid_params,n_jobs=-1,verbose=1)
clf.fit(X_train['processed_text'].values, y_train.values)
print("Best Score: ", clf.best_score_)
print("Best Params: ", clf.best_params_)

# cumulative logging report

In [32]:
INPUT_FOLDER_ID='172905132682'
logger.debug('Read the logging file from BOX location... ')

folder = client.folder(folder_id=INPUT_FOLDER_ID)
items = folder.get_items()
# make a list of iten name and item id of all items
files = [(item.name, item.id) for item in items]

# sort it by filename
files.sort(reverse=True, key=lambda x: x[0])
#print(files)
try:
    for i, f in enumerate(files):
        # get id of the latest file (based on timestamp)
        if f[0]=='l1triage_model_classifier_loggging.csv':
            file_id = f[1]
            file_info = client.file(file_id).content()
            report = pd.read_csv(io.StringIO(file_info.decode('utf-8')),index_col=0)
        else:
            report=pd.DataFrame()
            
    logger.debug('logging file  present...')
except Exception as e:
    print(e)
    logger.error('Getting no new logging file....', exc_info=True)

In [33]:
new_row_component={'Model_info':'component',
                   'Model_path':r'\\corp.uber.com\rpa\UiPath\Dev\IACOE OCR\models\L1Triage\Model\final_model',
                   'Lob':'L1_triage',
                   'Hyperparameters_used':model_component.steps,
                   'Validation_accuracy':result1['confusion_matrix'] [5:6].values,
                   'Model_status_flag':'active',
                   'Train_test_data_path':r'\\corp.uber.com\rpa\UiPath\Dev\IACOE OCR\models\L1Triage\Model\final_model\Data\df_traintestdata.csv',
                   'Deployed_date':pd.Timestamp("today").strftime("%d-%b-%Y")}

In [34]:
new_row_tier_type_1={'Model_info':'tier type 1',
                     'Model_path':r'\\corp.uber.com\rpa\UiPath\Dev\IACOE OCR\models\L1Triage\Model2\final_model',
                     'Lob':'L1_triage',
                     'Hyperparameters_used':model_tier_type_1.steps,
                     'Validation_accuracy':result2['confusion_matrix'] [5:6].values,
                     'Model_status_flag':'active',
                     'Train_test_data_path':r'\\corp.uber.com\rpa\UiPath\Dev\IACOE OCR\models\L1Triage\Model2\final_model\Data\df_traintestdata.csv',
                     'Deployed_date':pd.Timestamp("today").strftime("%d-%b-%Y")}

In [35]:
new_row_tier_type_2={'Model_info':'tier type 2',
                     'Model_path':r'\\corp.uber.com\rpa\UiPath\Dev\IACOE OCR\models\L1Triage\Model3\final_model',
                     'Lob':'L1_triage',
                     'Hyperparameters_used':model_tier_type_2.steps,
                     'Validation_accuracy':result3['confusion_matrix'] [5:6].values,
                     'Model_status_flag':'active',
                     'Train_test_data_path':r'\\corp.uber.com\rpa\UiPath\Dev\IACOE OCR\models\L1Triage\Model3\final_model\Data\df_traintestdata.csv',
                     'Deployed_date':pd.Timestamp("today").strftime("%d-%b-%Y")}

In [36]:
#append row to the dataframe
report = report.append(new_row_component, ignore_index=True)
report = report.append(new_row_tier_type_1, ignore_index=True)
report = report.append(new_row_tier_type_2, ignore_index=True)
report=report.loc[report.astype(str).drop_duplicates().index]

In [37]:
# Folder ID to upload the data
OUTPUT_FOLDER_ID=['170542616761','172902610098','172905132682']
# Create Filename based on timestamp
FILENAME ='l1triage_model_classifier_loggging.csv'
logging_path='./data'
logging_path_data = os.path.join(logging_path, FILENAME) 
report.to_csv(logging_path_data)
for i in OUTPUT_FOLDER_ID:
    config = JWTAuth.from_settings_file('Tax_Auto_Config.json')
    client = Client(config)
    # fetch all the items from the folder
    folder = client.folder(folder_id=i)
    items = folder.get_items()
    flag = False

    for item in items:
        # if file name exists, update the content
        if item.name == FILENAME:

            updated_file = client.file(item.id).update_contents(logging_path_data)
            print('File "{0}" has been updated'.format(updated_file.name))
            logger.debug('File "{0}" has been updated in BOX...'.format(updated_file.name))
            flag = True
            break

    # if file doesn not exists upload the file
    if not flag:
        uploaded_file = folder.upload(logging_path_data)
        print('File "{0}" has been uploaded'.format(uploaded_file.name))
        logger.debug('File "{0}" has been uploaded in BOX...'.format(uploaded_file.name))
logger.debug('Retraining completed sucessfully...')

File "l1triage_model_classifier_loggging.csv" has been uploaded
File "l1triage_model_classifier_loggging.csv" has been uploaded
File "l1triage_model_classifier_loggging.csv" has been uploaded


# Automated email notification

In [38]:
# datetime object containing current date and time
from datetime import datetime    
import pytz    
tz_NY = pytz.timezone('Asia/Kolkata')   
datetime_NY = datetime.now(tz_NY)  
ist=datetime_NY.strftime("%Y-%m-%d %H:%M:%S") 
message_success = {"personalizations": [
    {
    "to": [{
            "email": "amoham117@ext.uber.com"
        }, {
            "email": "nbandi@ext.uber.com"
        },
        {
            "email": "ssethu@ext.uber.com"
        },
         {
            "email": "rkunta@ext.uber.com"
        },
        {
            "email": "vbanap@ext.uber.com"
        },
        {
            "email": "pyagat@ext.uber.com"
        },
        {
            "email": "rsujay@ext.uber.com"
        }],
    "subject": "L1Triaging ML CICD automation successfull"
    }
],
"from": {
    "email": "ia-coe-support-group@uber.com"
},
    'content': [
        {
            'type': 'text/html',
            "value": "<html><head></head><body>Hello!<br><br>The L1triage Automation has been run successfully at "+str(ist)+" IST.<br><br>The output reports are ready for use https://uber.app.box.com/folder/170368651516.<br><br>You can reach out to ia-coe-support-group@uber.com for any questions or concerns.<br><br><br>Thanks,<br><b>Intelligent Automation COE Support Group<b></body></html>"

        }
]
}
try:
    response = sg.client.mail.send.post(request_body=message_success)
    print(response.status_code)
    logger.debug('L1 traige automated retraining email notification has been sent successfully...')
except Exception as e:
    print(str(e))
    logger.debug('Exception details::{}'.format(e))
if response.status_code == 202:
    print ("email sent")
else:
    print("Checking body",response.body)

202
email sent


In [39]:
end_time = time.time()
interval=time.strftime('%H:%M:%S', time.gmtime(end_time-start_time))
logger.debug('Total program execution time: {}\n\n\n\n'.format(interval))